# Creating a Table from Delta Lake

In this example notebook you will create a Table from data stored in the delta format.

In [ ]:
%pip install -r ../mltable-requirements.txt

In [ ]:
import mltable
from mltable import DataType

tbl = mltable.from_delta_lake(
    delta_table_uri="wasbs://data@azuremlexampledata.blob.core.windows.net/COVID-19_NYT/",
    timestamp_as_of="2023-01-01T00:00:00Z",
)

# format the date column
data_types = {"date": DataType.to_datetime(formats=["%Y-%m-%d"])}

# convert the date column to a date type
tbl = tbl.convert_column_types(data_types)

# show the first 5 records
tbl.show(5)

## 🐼 Load into Pandas

You can load the data into pandas using:

In [ ]:
df = tbl.to_pandas_dataframe()
df.head()

## 💾 Save data loading steps 
Next, you'll save all your data loading steps into an `MLTable` file. This allows you to *reproduce* your Pandas data frame at a later point in time without having to redefine the data loading steps in your code.

In [ ]:
# save the data loading steps in an MLTable file
tbl.save("./covid")

## 🤝 Create a data asset to aid sharing and reproducibility

You'll now create a data asset, which will automatically upload the `MLTable` to cloud storage (the default AzureML datastore) so that others can use it easily.

In [ ]:
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace = "<AML_WORKSPACE_NAME>"

In [ ]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
from azure.identity import DefaultAzureCredential

# connect to the AzureML workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

my_data = Data(
    path="./covid",
    type=AssetTypes.MLTABLE,
    description="COVID-19 cases in Washington county",
    name="covid-mltable-example",
)

ml_client.data.create_or_update(my_data)

## ♻️ Reproduce the table

You and team members can access this data asset and reproduce the table.

In [ ]:
import mltable
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# connect to the AzureML workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

# get the latest version of the data asset
data_asset = ml_client.data.list("covid-mltable-example").next()

# the table from the data asset id
tbl = mltable.load(f"azureml:/{data_asset.id}")

# load into pandas
df = tbl.to_pandas_dataframe()
df.head()